In [68]:
#if needed...

import import_ipynb
import Only_Pisa as pi

import copy

import networkx as nx
from cdlib import algorithms, readwrite, evaluation, viz, utils

from collections import defaultdict
from collections import OrderedDict
from collections import Counter
from operator import itemgetter
from itertools import groupby
import operator

import pandas as pd
import numpy
import json
import csv

from os import listdir
import os.path

import matplotlib.pyplot as plt
%matplotlib inline

In [69]:
node_labels = sorted([f for f in listdir('/home/salvatore/Scrivania/node_labels')])
projections = sorted([f for f in listdir('/home/salvatore/Scrivania/Projections')])

print(node_labels)
print(projections)

['0_2017_04.csv', '11_2017_03.csv', '13_2017_05.csv', '14_2016_11.csv', '15_2017_11.csv', '16_2016_10.csv', '17_2017_10.csv', '18_2016_09.csv', '19_2017_09.csv', '1_2016_08.csv', '2_2017_08.csv', '3_2016_12.csv', '4_2017_12.csv', '5_2017_02.csv', '6_2017_01.csv', '8_2017_07.csv', '9_2017_06.csv']
['0_Proj_Apr17.csv', '11_Proj_Mar17.csv', '13_Proj_May17.csv', '14_Proj_Nov16.csv', '15_Proj_Nov17.csv', '16_Proj_Oct16.csv', '17_Proj_Oct17.csv', '18_Proj_Sep16.csv', '19_Proj_Sep17.csv', '1_Proj_Aug16.csv', '2_Proj_Aug17.csv', '3_Proj_Dec16.csv', '4_Proj_Dec17.csv', '5_Proj_Feb17.csv', '6_Proj_Jan17.csv', '8_Proj_July17.csv', '9_Proj_Jun17.csv']


In [70]:
list_g = list()
for i, d in enumerate(projections):
    g = nx.Graph()
    
    with open('/home/salvatore/Scrivania/Projections/' + d,'rt')as f:
        data = csv.reader(f)
        for l in data:
            l = [s.split() for s in l]
            g.add_edge(l[0][0],l[0][1])
            
    list_g.append(g)

In [71]:
#GRAPHS ARE ORDERED
for i, nl in enumerate(node_labels):
    with open('/home/salvatore/Scrivania/node_labels/' + nl,'rt')as f:
        data = csv.reader(f)
        for l in data:
            if l[0] in list_g[i].nodes():
                list_g[i].add_node(l[0], labels=l[1])

In [78]:
for i, g in enumerate(list_g):
    
    path = "/home/salvatore/Scrivania/Communities_Proj/" + projections[i]
    os.mkdir(path)
    
    temp_g=list()
    for k, v in g.nodes.data():
        if len(v)!=0:
            temp_g.append(k)
            
    new_g = nx.subgraph(g, temp_g)
    
    mapping = dict(zip(new_g, range(0, len(new_g))))
    inv_map = {v: k for k, v in mapping.items()}
    g_relabel = nx.relabel_nodes(new_g, mapping)
    #print(g_relabel.number_of_nodes())
    
    for alpha in [0, 0.7, 0.8, 0.9, 1]:
        part, com_labels = pi.best_partition(g_relabel, alpha=alpha, beta=1-alpha)
        
        modularity = pi.modularity(part,g_relabel)
        print(modularity)
        purity = pi.purity(com_labels)
        
        sorted_part = sorted(list(part.items()), key=itemgetter(1))
        groups = groupby(sorted_part, key=itemgetter(1))
        group_by_com_attr = [[x[0] for x in v] for k, v in groups]

        
        original_group = list()
        for el in group_by_com_attr:
            el = copy.deepcopy(el)
            el = nx.subgraph(comp_0_rel, el)
            el = nx.relabel_nodes(el, inv_map)
            original_group.append(list(el.nodes()))
        
        coms = defaultdict(int)
        for n, c in part.items():
            coms[c] += 1
            
        #print(sorted(dict(coms).items(), key=lambda x: x[0]))
        attr = sorted(com_labels.items(), key=lambda x: x[0])
        
        max_attrs = list()
        for el in attr:
            max_attrs.append( [el[0], max(el[1].items(), key=operator.itemgetter(1))[0]] )
        
        res = {"coms": original_group,
           "alpha": alpha,
           "purity": purity,
           "modularity": modularity,
           "coms_max_label": max_attrs}
        
        js = json.dumps(res)
        filename = "alpha_" + str(alpha)
        f = open(os.path.join(path, filename), 'w')
        f.write(js)
        f.close()

1044
0.33584956689242385
0.33546754695618836
0.33479752125888396
0.3341228276534361
0.1780475424335987
1675
0.4999511196258057
0.4987739767559242
0.49982137734431925
0.49861218395852513
0.33505613311158255
627
0.1557314248854349
0.1542390564827562
0.15548919329380645
0.15452854704854155
0.09069696988586212
134
0.15307958477508637
0.15078770339443498
0.15307958477508637
0.1451566587548971
0.08584517715691048
827
0.41510291630889173
0.40789372001979257
0.41196575474559766
0.4013096532445257
0.15545648132889583
49
0.09777992719946893
0.09777992719946893
0.09650291316590741
0.09269834138412583
0.07866334905336417
905
0.3633567249959348
0.35866412776444445
0.3591002858877366
0.35811763527463997
0.10318928319874363
47
0.19345989207053155
0.19345989207053155
0.1829613414972898
0.16048394946830602
0.07170567021445816
951
0.43811509872663823
0.43744542437901734
0.43703865335264874
0.43686147917069446
0.21735009555445062
32
0.13750737898465165
0.13735979929161746
0.11207096474953618
0.1078912970

In [88]:
#list_g = list()
for d in sorted(date):
    g = nx.Graph()
    
    path = "/home/salvatore/Scrivania/Communities_Proj/Communities_Proj_" + d 
    os.mkdir(path)
    
    with open('/home/salvatore/Scrivania/TNA-July19-master/Projections/Proj_' + d + '.csv','rt')as f:
        data = csv.reader(f)
        for l in data:
            l = [s.split() for s in l]
            g.add_edge(l[0][0],l[0][1])
    
    with open('/home/salvatore/Scrivania/TNA-July19-master/labeled_net/MaxT' + d + '.csv','rt')as f:
        data = csv.reader(f)
        for l in data:
            if l[0] in g.nodes():
                g.add_node(l[0], labels=l[1])
                
    comps_list = list(nx.connected_components(g))
    #for el in comps_list:
    #    print(len(el))
    comp_0 = nx.subgraph(g, comps_list[0])
    
    mapping = dict(zip(comp_0, range(0, len(comp_0))))
    inv_map = {v: k for k, v in mapping.items()}
    comp_0_rel = nx.relabel_nodes(comp_0, mapping)

    #list_g.append(g)
    
    for alpha in [0, 0.7, 0.8, 0.9, 1]:
        part, com_labels = pi.best_partition(comp_0_rel, alpha=alpha, beta=1-alpha)
        
        modularity = pi.modularity(part,comp_0_rel)
        purity = pi.purity(com_labels)
        
        sorted_part = sorted(list(part.items()), key=itemgetter(1))
        groups = groupby(sorted_part, key=itemgetter(1))
        group_by_com_attr = [[x[0] for x in v] for k, v in groups]

        
        original_group = list()
        for el in group_by_com_attr:
            el = copy.deepcopy(el)
            el = nx.subgraph(comp_0_rel, el)
            el = nx.relabel_nodes(el, inv_map)
            original_group.append(list(el.nodes()))
        
        coms = defaultdict(int)
        for n, c in part.items():
            coms[c] += 1
            
        #print(sorted(dict(coms).items(), key=lambda x: x[0]))
        attr = sorted(com_labels.items(), key=lambda x: x[0])
        
        max_attrs = list()
        for el in attr:
            max_attrs.append( [el[0], max(el[1].items(), key=operator.itemgetter(1))[0]] )
        
        res = {"coms": original_group,
           "alpha": alpha,
           "purity": purity,
           "modularity": modularity,
           "coms_max_label": max_attrs}
        
        js = json.dumps(res)
        filename = "alpha_" + str(alpha)
        f = open(os.path.join(path, filename), 'w')
        f.write(js)
        f.close()

FileExistsError: [Errno 17] File exists: '/home/salvatore/Scrivania/Communities_Proj/Communities_Proj_June16'

In [74]:
for g in list_g:
    comps_list = list(nx.connected_components(g))
    comp_0 = nx.subgraph(g, comps_list[0])
    
    mapping = dict(zip(g, range(0, len(g))))
    g = nx.relabel_nodes(g, mapping)
    
    for alpha in [0, 0.7, 0.8, 0.9, 1]:
        part, com_labels = pi.best_partition(g, alpha=alpha, beta=1-alpha)

NameError: name 'list_g' is not defined

In [56]:
comps_list = list(nx.connected_components(list_g[0]))
print(len(comps_list))
comp_0 = nx.subgraph(list_g[0], comps_list[0])

mapping = dict(zip(comp_0, range(0, len(comp_0))))
g_relab = nx.relabel_nodes(list_g[0], mapping)

for alpha in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
    part, com_labels = pi.best_partition(g_relab, alpha=alpha, beta=1-alpha)
    
    sorted_part = sorted(list(part.items()), key=itemgetter(1))
    groups = groupby(sorted_part, key=itemgetter(1))
    group_by_com_attr = [[x[0] for x in v] for k, v in groups]
    
    res = {"communities": group_by_com_attr,
           "algorithm": "null",
           "params": "null",
           "overlap": "null",
           "coverage": "null"}
    
    js = json.dumps(res)
    filename = str(count) + "_alpha_" + str(alpha) + "_beta_" + str(alpha-1)
    count+=1
    f = open(os.path.join(path, filename), 'w')
    f.write(js)
    f.close()

1


/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyError: 469